# **Procura de Problemas Éticos em Issues**

In [131]:
import pandas as pd
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

## **Fazendo a leitura dos dados:**

In [132]:
issues = pd.read_csv("out/daggerfall-unity-issues-3.csv")
comentarios = pd.read_csv("out/daggerfall-unity-comentarios-3.csv")

# Concatenando o Titulo e a Descrição das issues
issues['Info'] = issues['TituloIssue'] + issues['DescricaoIssue']

# Convertendo todos os tipos para String
issues['Info'] = issues['Info'].astype(str)
comentarios['Comentario'] = comentarios['Comentario'].astype(str)

## **Tokenizando os Comentarios, Titulo e Descrição das Issues**

In [140]:
# Definindo as StopWords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

def removerStopWords(palavras):
    return [palavra for palavra in palavras if palavra not in stopWords and palavra.isalpha()]

# Criando um tokenizer que considera expressões
expressoes = {'Data Governance', 'Data Protection','Informed Consent', 'Lack of Data', 'User Data Collection' 'Business Ethics','Conflict of Interest',
            'Human Agency', 'Intellectual Property','Regulatory Approaches', 'Self-Conception','Professional Ethics', 'Work Ethics','Common Goods','Individual Differences',
            'Non-Discrimination', 'Non-Maleficence','Prevention of Harm', 'Quality of Life', 'Respect for Human Autonomy', 'Retention and Addiction', 'Social Justice',
            'Speech Issues', 'Technical Robustness','Computer Abuse', 'Malicious Use'}

expressaoRegular = r'\b(?:' + '|'.join(expressoes) + r')\b|\w+'
tokenizer = RegexpTokenizer(expressaoRegular)


# Tokenize dos comentarios das issues
comentariosToK = []
for i in range (comentarios['Comentario'].size):
    comentariosToK.append(removerStopWords(tokenizer.tokenize(comentarios.loc[i]['Comentario'])))
    removerStopWords(comentariosToK[i])

# Tokenize do campo Info
tituloDescricaoToK = []
for k in range(issues['Info'].size):
    tituloDescricaoToK.append(removerStopWords(tokenizer.tokenize(issues.loc[k]['Info'])))


[nltk_data] Downloading package stopwords to /home/zoega/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'This'

## **Contando a ocorrência de problemas éticos:**

In [134]:
# Definindo os EthicalIssues
EthicalIssues = ['Bias', 'Data Governance', 'Data Protection', 'Encryption', 'Informed Consent', 'Lack of Data', 'Monetization', 'Openness', 'Privacy', 'User Data Collection'
            'Authorship', 'Autonomy', 'Beneficence', 'Business Ethics', 'Commerce', 'Compliance', 'Confidentiality', 'Conflict of Interest', 'Context', 'Dependability',
            'Fairness', 'Human Agency', 'Intellectual Property', 'Oversight', 'Regulatory Approaches', 'Responsibility', 'Trust', 'Trustworthiness', 'Axiology', 'Freedom',
            'Self-Conception', 'Solidarity', 'Utility', 'Care', 'Competence', 'Professional Ethics', 'Work Ethics', 'Access', 'Accessibility', 'Common Goods', 'Dignity',
            'Diversity', 'Equality', 'Equity', 'Humanity', 'Inclusiveness', 'Individual Differences', 'Inequality', 'Justice', 'Non-Discrimination', 'Non-Maleficence',
            'Participation', 'Plurality', 'Prevention of Harm', 'Quality of Life', 'Respect for Human Autonomy', 'Retention and Addiction', 'Social Justice', 'Sustainability',
            'Unemployment', 'Welfare', 'Accountability', 'Accuracy', 'Anonymity', 'Comprehensibility', 'Consistency', 'Contestability', 'Explainability', 'Explicability',
            'Integrity', 'Interpretability', 'Liability', 'Reliability', 'Safety', 'Security', 'Speech Issues', 'Technical Robustness', 'Traceability', 'Transparency',
            'Usability', 'Computer Abuse', 'Malicious Use']

# Criando o DataFrame que guardará as ocorrências dos comentários
ocorrenciasComents = pd.DataFrame()
linhaInicial = 0 * len(EthicalIssues)
palavrasAntes = "" * len(EthicalIssues)
palavrasDepois = "" * len(EthicalIssues)

ocorrenciasComents['EthicalIssues'] = EthicalIssues
ocorrenciasComents['Ocorrencias'] = linhaInicial
ocorrenciasComents["PalavrasAntes"] = palavrasAntes
ocorrenciasComents["PalavrasDepois"] = palavrasDepois
ocorrenciasComents = ocorrenciasComents.set_index('EthicalIssues')
ocorrenciasComents

# Criando o DataFrame que guardará as ocorrências do título e descrição
ocorrenciasTituloDescricao = pd.DataFrame()
linhaInicial = 0 * len(EthicalIssues)
palavrasAntes = "" * len(EthicalIssues)
palavrasDepois = "" * len(EthicalIssues)

ocorrenciasTituloDescricao['EthicalIssues'] = EthicalIssues
ocorrenciasTituloDescricao['Ocorrencias'] = linhaInicial
ocorrenciasTituloDescricao["PalavrasAntes"] = palavrasAntes
ocorrenciasTituloDescricao["PalavrasDepois"] = palavrasDepois
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.set_index('EthicalIssues')
ocorrenciasTituloDescricao

,Ocorrencias,PalavrasAntes,PalavrasDepois
EthicalIssues,,,
Bias,0,,
Data Governance,0,,
Data Protection,0,,
Encryption,0,,
Informed Consent,0,,
...,...,...,...
Traceability,0,,
Transparency,0,,
Usability,0,,


In [135]:
for i in range(len(comentariosToK)):
    for j in range(len(comentariosToK[i])):
        
        if(comentariosToK[i][j] in EthicalIssues):
            
            ocorrenciasComents.loc[comentariosToK[i][j], 'Ocorrencias'] += 1
            
            if((j - 1) >= 0):
                if(ocorrenciasComents.loc[comentariosToK[i][j],'PalavrasAntes'] == ''):
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasAntes'] = comentariosToK[i][j - 1]
                else:
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasAntes'] += ('\n' + comentariosToK[i][j - 1])
            if((j + 1) <= (len(comentariosToK[i]) - 1)):
                if(ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] == ''):
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] = comentariosToK[i][j + 1]
                else:
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] += ('\n' + comentariosToK[i][j + 1])


for i in range(len(tituloDescricaoToK)):
    for j in range(len(tituloDescricaoToK[i])):
        
        if(tituloDescricaoToK[i][j] in EthicalIssues):
            
            ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'Ocorrencias'] += 1
            
            if((j - 1) >= 0):
                if(ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j],'PalavrasAntes'] == ''):
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasAntes'] = tituloDescricaoToK[i][j - 1]
                else:
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasAntes'] += ('\n' + tituloDescricaoToK[i][j - 1])
            if((j + 1) <= (len(tituloDescricaoToK[i]) - 1)):
                if(ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] == ''):
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] = tituloDescricaoToK[i][j + 1]
                else:
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] += ('\n' + tituloDescricaoToK[i][j + 1])

# Isso aqui funcionou, porém ainda não consegui resolver os problemas de links e imagens.

### **Exportando os resultados para um csv:**

In [136]:
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.reset_index()
ocorrenciasComents = ocorrenciasComents.reset_index()

caminhoPasta = 'out'
nomeArquivo = 'Comentarios.csv'
caminhoArquivo = os.path.join(caminhoPasta, nomeArquivo)

if not os.path.exists(caminhoPasta):
    os.makedirs(caminhoPasta)

ocorrenciasComents.to_csv(caminhoArquivo, sep=',', index=False, header=True, na_rep='N/A', encoding='utf-8')

caminhoPasta = 'out'
nomeArquivo = 'TituloDescricao.csv'
caminhoArquivo = os.path.join(caminhoPasta, nomeArquivo)

if not os.path.exists(caminhoPasta):
    os.makedirs(caminhoPasta)

ocorrenciasTituloDescricao.to_csv(caminhoArquivo, sep=',', index=False, header=True, na_rep='N/A', encoding='utf-8')